In [28]:
### Author: Md Shadman Sakib 4/1/2024
### Input: .csv files of each observation station having observation data and simulation output
### It reads in the file of each station, calculate the max and min in each of the timestep and then plots the ensemble. 
### Control: Can define how many simulation can be plotted.
### To be developped: Control over the simulation that can be plotted
### Output: Subplot timeseries graphs for each individual stations
### Ensemble plot
### Output: 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import r2_score
from pandas.io.formats.style import Styler
import matplotlib.dates as mdates
import glob
%matplotlib qt

def evaluate(obs_wl, sim_wl):
    from sklearn.metrics import r2_score
    import numpy as np
    import pandas as pd
    
    # Calculate coefficient of determination (true vs. predicted)
    cd = r2_score(obs_wl, sim_wl)
    # Calculate correlation matrix (R)
    R = np.corrcoef(sim_wl, obs_wl)  
    # Calculate the mean absolute error (MAE)
    res = sim_wl - obs_wl
    res_abs = res.abs()
    MAE = res_abs.mean()
    # Calculate the mean square error (MSE)
    res_pow = res.pow(2)
    MSE = res_pow.mean()
    # Calculate the root-mean square error (RMSE)
    RMSE = np.sqrt(MSE)
    # Calculate Kling-Gupta Efficiency
    sd_y_sim = np.std(sim_wl)
    sd_y_obs = np.std(obs_wl)
    avr_y_sim = np.mean(sim_wl)
    avr_y_obs = np.mean(obs_wl)
    alpha=sd_y_sim/sd_y_obs
    beta=avr_y_sim/avr_y_obs
    KGE = 1 - np.sqrt(np.square(R[0,1]-1) + np.square(alpha-1) + np.square(beta-1))
    # Calculate Nash-Sutcliffe Efficiency
    res2 = obs_wl - np.mean(obs_wl)
    res2_pow = res2.pow(2)
    NSE = 1 - (res_pow.sum()/res2_pow.sum())
    return np.round(np.array([MAE, RMSE, NSE, KGE]),2)


station_dict = {
    "SP": "Sewells Point, VA",
    "MP": "Money Point, VA",
    "KK": "Kiptopeke, VA",
    "WAC": "Wachapreague, VA",
    "YOR": "Yorktown USCG, VA",
    "WP": "Windmill Point, VA",
    "OC": "Ocean City Inlet, MD",
    "LEW": "Lewisetta, VA",
    "SI": "Solomons Island, MD",
    "DC": "Washington, DC",
    "BH": "Bishops Head, MD",
    "CC": "Chesapeake City, MD",
    "BAL": "Baltimore, MD",
    "TB": "Tolchester Beach, MD",
    "ANN": "Annapolis, MD",
    "CAM": "Cambridge, MD",
    "DAH": "Dahlgren, VA",
    "CBBT": "CBBT, Chesapeake Channel, VA"
}

#-------------------------------INPUT START---------------------------------------------------------------#

local_path_set=r'E:\VirginiaTech-Research\ChesapeakeModel\Tidal_Calibration\Results'
data_paths=glob.glob(local_path_set+"\*.csv")
optimal_sim_id=5
plt.rcParams.update({'font.size': 10.5})

#-------------------------------INPUT END-----------------------------------------------------------------#
#---------------------------------------------------------------------------------------------------------#
#-------------------------------DATA PROCESSING START-----------------------------------------------------#


station_count=len(data_paths)

num_rows = 6 ## dc is currently missing
num_cols = 2

fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 15))
fig.subplots_adjust(top=0.9) 


# Initialize subplot indices
row_index = 0
col_index = 0


for data_path in data_paths:
    data = pd.read_csv(data_path)
    station=data_path.split('\\')[-1].split('.')[0]
    data['DateTime (GMT)'] = data['DateTime (GMT)'].astype('datetime64[ns]')
    sim_columns = data.columns[2:]
    # for column in df_cal.columns[1:]:
        # axs[row_index, col_index].plot(df_cal['DateTime (GMT)'], df_cal[column], label=column)
    
    ensemble_df = pd.DataFrame({
    'DateTime (GMT)': data['DateTime (GMT)'],
    'NOAA WL (m)' : data['NOAA WL (m)'],
    'optimal': data['sim'+str(optimal_sim_id)+'_'+station], # this value has to be substituted with the best-fit model: DelftFMOutput.nc2ObsStation.csv-obsimportedCPBFormat
    'ensemble_max': data[sim_columns].max(axis=1),
    'ensemble_min': data[sim_columns].min(axis=1)
    })

    # Plot the observed data
    axs[row_index, col_index].plot(ensemble_df['DateTime (GMT)'], ensemble_df['NOAA WL (m)'], 'r.', label='Observation', markersize=2)
    # Plot the ensemble mean with a shaded area for the ensemble spread
    axs[row_index, col_index].plot(ensemble_df['DateTime (GMT)'], ensemble_df['optimal'], 'b-', label='Optimal')
    axs[row_index, col_index].fill_between(ensemble_df['DateTime (GMT)'], ensemble_df['ensemble_min'], ensemble_df['ensemble_max'], color='gray', alpha=0.5, label='Ensemble')

    ## titl: setting up/annotating the evaluation metrics on to the graphs
    obs_wl=ensemble_df['NOAA WL (m)']
    sim_wl=ensemble_df['optimal']
    
    eval_met=evaluate(obs_wl, sim_wl) # [MAE, RMSE, NSE, KGE]


    # Adjusting the top margin to create more space for the annotation
    station_title=station_dict[station]+' ['+station+']'
    title_met='MAE: '+str(eval_met[0])+'m; '+' RMSE: '+str(eval_met[1])+'m; '+' NSE: '+str(eval_met[2])+'; '+' KGE: '+str(eval_met[3]) 
    

    # Adjust y limits to create space for the annotation
    current_ylim = axs[row_index, col_index].get_ylim()
    axs[row_index, col_index].set_ylim(current_ylim[0], current_ylim[1] + 0.125 * (current_ylim[1] - current_ylim[0]))
    
    axs[row_index, col_index].text(0.95, 0.95, station_title, transform=axs[row_index, col_index].transAxes, fontsize=10.5, verticalalignment='top', horizontalalignment='right')
    axs[row_index, col_index].text(0.05, 0.95, title_met, transform=axs[row_index, col_index].transAxes, fontsize=10.5, verticalalignment='top', horizontalalignment='left')
    
    #axs[row_index, col_index].set_title(title_met)
    axs[row_index, col_index].set_ylabel('Tidal level [m]')

    if row_index != num_rows - 1:
        print(row_index)
        print('ok')
        # axs[row_index, col_index].set_xticklabels([]) # this command removes the x-axis values
        axs[row_index, col_index].xaxis.set_major_formatter(mdates.DateFormatter('%d/%b'))
        axs[row_index, col_index].xaxis.set_major_locator(mdates.DayLocator(interval=4))

    else:
    
        axs[row_index, col_index].xaxis.set_major_formatter(mdates.DateFormatter('%d/%b'))
        axs[row_index, col_index].xaxis.set_major_locator(mdates.DayLocator(interval=4))

    ### letting only 1 legend to remain.
    if row_index == 0 and col_index == 0:
        axs[row_index, col_index].legend(loc='lower right')



    # Hide x-axis labels for all but the last row subplots


    
    # Update subplot indices
    if col_index == num_cols - 1:
        row_index += 1
        col_index = 0
    else:
        col_index += 1

 
    
    del ensemble_df # deleting the ensemble

plt.tight_layout()
print('[MAE, RMSE, NSE, KGE]')

0
ok
0
ok
1
ok
1
ok
2
ok
2
ok
3
ok
3
ok
4
ok
4
ok
[MAE, RMSE, NSE, KGE]


In [10]:
station_dict = {
    "SP": "Sewells Point, VA",
    "MP": "Money Point, VA",
    "KK": "Kiptopeke, VA",
    "WAC": "Wachapreague, VA",
    "YOR": "Yorktown USCG Training Center, VA",
    "WP": "Windmill Point, VA",
    "OC": "Ocean City Inlet, MD",
    "LEW": "Lewisetta, VA",
    "SI": "Solomons Island, MD",
    "DC": "Washington, DC",
    "BH": "Bishops Head, MD",
    "CC": "Chesapeake City, MD",
    "BAL": "Baltimore, MD",
    "TB": "Tolchester Beach, MD",
    "ANN": "Annapolis, MD",
    "CAM": "Cambridge, MD",
    "DAH": "Dahlgren, VA",
    "CBBT": "CBBT, Chesapeake Channel, VA"
}
station_dict[station]

'Yorktown USCG Training Center, VA'

In [7]:
station

'YOR'